**<center> <span style="font-family:serif; font-size:34px;"> CSE 253 Assignment 2</span> </center>**

<style>
body {
  font-family: 'Times New Roman', sans-serif;
  font-size: 20pt;
}
h1 { font-weight: bold !important; }
p {
  font: 16pt 'Times New Roman' !important;
}

div.text_cell_render h1, .rendered_html h1 {
  color: #126dce !important;
  font-size: 2.2em !important;
  font-weight: bold !important;
  margin: 0.4em 0 0.3em 0 !important;
}
div.text_cell_render h2, .rendered_html h2 {
  color: #126dce !important;
  font-size: 1.7em !important;
  font-weight: bold !important;
  margin: 0.3em 0 0.2em 0 !important;
}
div.text_cell_render h3, .rendered_html h3 {
  color: #126dce !important;
  font-size: 1.35em !important;
  font-weight: bold !important;
  margin: 0.3em 0 0.1em 0 !important;
}
div.text_cell_render h4, .rendered_html h4 {
  color: #126dce !important;
  font-size: 1.1em !important;
  font-weight: bold !important;
}
div.text_cell_render h5, .rendered_html h5 {
  color: #126dce !important;
  font-size: 1em !important;
  font-weight: bold !important;
}

/* Markdown 代码块（行内、块） */
div.text_cell_render pre, div.text_cell_render code,
.rendered_html pre, .rendered_html code {
  font-family: 'Fira Mono', 'Consolas', monospace !important;
  font-size: 13pt !important;
  color: #353535 !important;
  background: #efefef !important;
  border-radius: 2px !important;
  border-left: 3px solid rgba(240,147,43,.50) !important;
  padding: 5px 10px !important;
  margin: 6px 0 !important;
  line-height: 1.6 !important;
  max-width: 90%;
}

/* Markdown 内联代码高亮（如 `code`） */
div.text_cell_render code, .rendered_html code {
  background: #efefef !important;
  color: #d6336c !important;
  padding: 2px 5px !important;
  border-radius: 2px !important;
  font-size: 1em !important;
  font-family: 'Fira Mono', 'Consolas', monospace !important;
}

/* 表格美化 */
.rendered_html table, div.text_cell_render table {
  font-family: 'Times New Roman', sans-serif !important;
  font-size: 1em !important;
  color: #353535 !important;
  background: #fff !important;
  border-collapse: collapse !important;
  border: 1px solid #e7e7e7 !important;
  margin: 1em 0 !important;
  width: auto !important;
}
.rendered_html th, .rendered_html td, div.text_cell_render th, div.text_cell_render td {
  border: 1px solid #e7e7e7 !important;
  padding: 8px 12px !important;
  text-align: left !important;
  vertical-align: middle !important;
}
.rendered_html thead, div.text_cell_render thead {
  background: #ebebeb !important;
  color: #126dce !important;
  font-weight: bold !important;
}
.rendered_html tbody tr:nth-child(odd), div.text_cell_render tbody tr:nth-child(odd) {
  background: #fafafa !important;
}
.rendered_html tbody tr:hover, div.text_cell_render tbody tr:hover {
  background: #e6f7ff !important;
}

/* 列表、段落间距微调 */
.rendered_html * + p {
  margin-top: 0.5em !important;
  margin-bottom: 0.5em !important;
}

/* 让无序和有序列表更紧凑 */
.rendered_html ul, .rendered_html ol, 
div.text_cell_render ul, div.text_cell_render ol {
    margin-top: 0.08em !important;
    margin-bottom: 0.08em !important;
    padding-left: 2em !important;
}

.rendered_html ul li, .rendered_html ol li,
div.text_cell_render ul li, div.text_cell_render ol li {
    margin-top: 0 !important;
    margin-bottom: 0 !important;
    padding-top: 0 !important;
    padding-bottom: 0 !important;
    line-height: 1.05 !important;   /* 极致紧凑 */
}
</style>


# <center> <span style="font-family:semi-serif; font-size:28px;"> Overview</span></center>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import miditoolkit
import pretty_midi
from midi2audio import FluidSynth
from IPython.display import Audio

# <center> <span style="font-family:semi-serif; font-size:28px;"> Task 1: Unconditional Symbolic Music Generation</span></center>


This notebook only covers key portions of our implementation. For complete implementation and other scripts used in this project, refer to this anonymous repository: [https://anonymous.4open.science/r/remi-pytorch-6C7D](https://anonymous.4open.science/r/remi-pytorch-6C7D).

## <span style="font-family:serif; font-size:28px;"> 1. Introduction</span>


## <span style="font-family:serif; font-size:24px;"> 1.1 Task and Context</span>


<!-- Where does your dataset come from? What is it for, how was it collected. etc -->

**Where is the dataset from**: In our project, we used the pop piano music dataset **POP1K7** published by [Hsiao et al. 2021](https://arxiv.org/pdf/2101.02402). This dataset contains 1,748 pop piano tracks, 108 hours in total, all in 4/4 time signature, originally collected from the Internet.

**What is the dataset for**: The dataset is used for automatic **Pop piano composition (unconditional symbolic music generation)**.


## <span style="font-family:serif; font-size:24px;"> 1.2 Data Preprocessing and Loading</span>


The dataset is utilized in its processed form as provided by the original authors:

- **Transcription**: The state-of-the-art RNN model for automatic piano transcription, ["Onset and Frames" (Hawthorne et al. 2018a)](https://arxiv.org/abs/1710.11153), is employed to estimate the pitch, onset and offset times, as well as the velocity of musical notes from audio.

- **Synchronization**: To convert wall clock time into symbolic timing, the RNN-based model from the Python package [madmom (Bock et al. 2016)](https://arxiv.org/abs/1605.07008) is used to estimate downbeat and beat positions, representing the state-of-the-art for this task. Subsequently, 480 ticks are interpolated between each pair of adjacent beats, and absolute time is mapped to the corresponding tick. This approach preserves fine timing accuracy. Tempo changes are inferred based on the intervals between consecutive beats.

- **Analysis**: For the conditional generation task, melody notes and chord symbols are estimated from the transcription results to construct lead sheets. A rule-based chord recognition algorithm is developed to identify 12 chord roots and 7 chord qualities. The “Skyline algorithm” is applied to extract melodies. Since lead sheets are typically in a coarser time resolution, both chord symbols and melody notes are quantized to quarter notes (i.e., beat times).

- **Quantization**: Tempo, velocity, duration, and beat positions are quantized to reduce the overall vocabulary size.


<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/imgs/data_proc_diagram.png"/> 

*Figure 1: POP1K7 Data Processing Pipeline, originally from [Compound Word Transformer
](https://github.com/YatingMusic/compound-word-transformer/blob/main/dataset/Dataset.md)*

## <span style="font-family:serif; font-size:24px;"> 1.3 Exploratory Analysis</span>


We analyze several musical features for each MIDI file, including:

- **Pitch density**: the ratio of unique pitches to the total number of notes, reflecting melodic diversity.

- **Pitch range**: the difference between the highest and lowest note, indicating the span of pitches used.

- **Note density**: the number of notes per second, representing how densely notes are distributed in time.

- **Average pitch interval**: the mean distance between consecutive note pitches, describing the typical melodic movement.

- **Average inter-onset interval (IOI)**: the average time gap between the start times of adjacent notes, characterizing the rhythmic pacing.

These features together capture key aspects of a piece’s melodic and rhythmic structure, providing a quantitative summary of its musical characteristics.

The table below presents the statistics of the dataset.

| Feature             | Mean     | Median   | Min     | Max     | Std      | N    |
|---------------------|----------|----------|---------|---------|----------|------|
| pitch_density       | 0.0319   | 0.0288   | 0.0081  | 0.1395  | 0.0141   | 1747 |
| pitch_range         | 54.4648  | 53.0000  | 28.0000 | 80.0000 | 8.9902   | 1747 |
| avg_pitch_interval  | 13.0434  | 12.8085  | 6.9252  | 21.7081 | 2.1046   | 1747 |
| note_density        | 6.0185   | 5.6158   | 2.0360  | 20.4708 | 2.1945   | 1747 |
| avg_ioi             | 0.1823   | 0.1742   | 0.0476  | 0.4813  | 0.0594   | 1747 |


<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/pop1k7_analysis/avg_ioi.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/pop1k7_analysis/avg_pitch_interval.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/pop1k7_analysis/note_density.png" width="350"/>
<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/pop1k7_analysis/pitch_density.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/pop1k7_analysis/pitch_range.png" width="350"/> 

*Figure 2: Music Feature Distribution of POP1K7*

The key portions of data analysis script is attach below, which implemented music feature extraction:

```python
def extract_features(pm: pretty_midi.PrettyMIDI) -> Dict[str, float]:
    notes = [note for instr in pm.instruments for note in instr.notes if not instr.is_drum]
    if not notes:
        return None

    pitches = [note.pitch for note in notes]
    pitch_count = len(set(pitches))
    pitch_range = max(pitches) - min(pitches)
    pitch_intervals = [abs(pitches[i] - pitches[i - 1]) for i in range(1, len(pitches))]
    avg_pitch_interval = np.mean(pitch_intervals) if pitch_intervals else 0

    onset_times = sorted([note.start for note in notes])
    ioi = [onset_times[i] - onset_times[i - 1] for i in range(1, len(onset_times))]
    avg_ioi = np.mean(ioi) if ioi else 0

    start_time = min([note.start for note in notes])
    end_time = max([note.end for note in notes])
    duration = end_time - start_time
    duration = duration if duration > 0 else 1  # avoid div by zero

    note_density = len(notes) / duration
    pitch_density = pitch_count / len(notes) if len(notes) > 0 else 0

    return {
        "pitch_density": pitch_density,
        "pitch_range": pitch_range,
        "avg_pitch_interval": avg_pitch_interval,
        "note_density": note_density,
        "avg_ioi": avg_ioi
    }
```

# <span style="font-family:serif; font-size:28px;"> 2. Modeling</span>


## <span style="font-family:serif; font-size:24px;"> 2.1 ML Formulation</span>


We formulate expressive pop piano music generation as an **autoregressive sequence modeling problem** over a discrete vocabulary of symbolic music events.

Let  
- $\mathcal{V}$ denote the vocabulary of REMI events (Bar, Position, Note-On, Note Duration, Note Velocity, Tempo, Chord, etc.);
- a song is represented as an event sequence $\mathbf{x} = (x_1, x_2, ..., x_T)$, where $x_t \in \mathcal{V}$.

The goal is to model the joint probability of the sequence:
$$
P(\mathbf{x}) = P(x_1, x_2, ..., x_T)
$$
which, by the chain rule, is factorized as:
$$
P(\mathbf{x}) = \prod_{t=1}^T P(x_t \mid x_1, ..., x_{t-1})
$$

A neural network is trained to estimate the conditional probability:
$$
P_\theta(x_t \mid x_1, ..., x_{t-1})
$$
where $\theta$ denotes the model parameters.

During training, the objective is to minimize the negative log-likelihood (or cross-entropy loss):
$$
\mathcal{L}(\theta) = -\sum_{t=1}^T \log P_\theta(x_t \mid x_1, ..., x_{t-1})
$$

**Generation:**  
At inference time, given a (possibly empty) seed sequence, the model generates a new composition by sampling each event $x_t$ autoregressively from the learned distribution until an end-of-piece token or a maximum length is reached.


## <span style="font-family:serif; font-size:24px;"> 2.2 Discussion on Model Selection</span>


<img src="https://miditok.readthedocs.io/en/latest/_images/remi.png"/>

*Figure 3: The REMI event representation.*

In our experiments, we adopt Transformer-based models as the primary approach for symbolic music generation, using a [**REMI**](https://arxiv.org/abs/2002.00212) tokenization scheme to represent musical events. For comparison, we implement Markov chain and LSTM-based models as baseline methods.

**Markov Chain models**:
<ul class="key-features"> 
    <li><strong>Advantages:</strong> Simple and computationally efficient, making them well-suited as baseline models.</li> 
    <li><strong>Disadvantages:</strong> Unable to capture long-range dependencies or global musical structure, often resulting in repetitive and locally constrained outputs.</li> 
</ul>

**LSTM-based models**:
<ul class="key-features">
    <li><strong>Advantages:</strong> Maintain a recurrent hidden state, which allows them to capture longer-term dependencies and musical motifs.</li> 
    <li><strong>Disadvantages:</strong> Still limited in modeling very long-range structure. The sequential nature of LSTM computations also limits efficiency and scalability.</li> 
</ul>

**Transformer-based models**:
<ul class="key-features"> 
    <li><strong>Advantages:</strong> Leverage self-attention mechanisms to capture both local and global dependencies across a sequence, enabling the generation of music with coherent long-term structure. Their parallelizable architecture also makes them more scalable than RNN-based models.</li> 
    <li><strong>Disadvantages:</strong> Require significantly greater computational resources and are more complex to implement, especially for long sequences.</li> 
</ul>


## <span style="font-family:serif; font-size:24px;"> 2.3 Codes</span>


In this section, we walk through the key portions of our transformer-based method implementation.

### <span style="font-family:serif; font-size:20px;"> 2.3.1 Tokenization</span>


In our pipeline, we use the REMI tokenization scheme to convert raw MIDI files into discrete symbolic event sequences, which is then used by the transformer model.

Key steps:
- **Extracting events**: Each MIDI file is parsed and quantized into REMI events, optionally including chords.

- **Event-to-token mapping**: Every event is converted into an integer using the vocabulary dictionary.

- **Sequence segmentation**: Long token sequences are split into overlapping segments of fixed length (e.g., 512 tokens per segment), forming input-output pairs for training.


```python
class NewsDataset(Dataset):
    def __init__(self, train = True, midi_l = [], dict_pth = './dictionary/dictionary_REMI-tempo-checkpoint.pkl'):
        self.midi_l = midi_l
        self.tokenizer = REMI()
        self.checkpoint_path = dict_pth
        self.x_len = 512
        self.dictionary_path = dict_pth
        self.event2word, self.word2event = pickle.load(open(self.dictionary_path, 'rb'))
        self.parser = self.prepare_data(self.midi_l)
        self.train = train
    
    def __len__(self):
        return len(self.parser)
    
    def __getitem__(self, index):
        if self.train:
            return self.parser[index]
        else:
            return self.words
            
    def extract_events(self, input_path):
        note_items, tempo_items = utils.read_items(input_path)
        note_items = utils.quantize_items(note_items)
        max_time = note_items[-1].end
        if 'chord' in self.checkpoint_path:
            chord_items = utils.extract_chords(note_items)
            items = chord_items + tempo_items + note_items
        else:
            items = tempo_items + note_items
        groups = utils.group_items(items, max_time)
        events = utils.item2event(groups)
        return events
        
    def prepare_data(self, midi_paths):
        print(f'{len(midi_paths)} midis.')
        # extract events
        all_events = []
        for path in midi_paths:
            events = self.extract_events(path)
            all_events.append(events)
        print(f'{len(all_events)} events.')
        # event to word
        all_words = []
        for events in all_events:
            words = []
            for event in events:
                e = '{}_{}'.format(event.name, event.value)
                if e in self.event2word:
                    words.append(self.event2word[e])
                else:
                    # OOV
                    if event.name == 'Note Velocity':
                        words.append(self.event2word['Note Velocity_21'])
                    else:
                        print('something is wrong! {}'.format(e))
            all_words.append(words)
        # to training data
        print(f'{len(all_words)} words.')
        
        segments = []
        for words in all_words:
            if len(words) < self.x_len + 1:
                continue
            step_size = self.x_len // 2
            for i in range(0, len(words) - self.x_len, step_size):
                if i + self.x_len + 1 <= len(words):
                    x = words[i:i + self.x_len]
                    y = words[i + 1:i + self.x_len + 1]
                    segments.append([x, y])
        segments = np.array(segments)
        print(segments.shape)
        return segments
```

### <span style="font-family:serif; font-size:20px;"> 2.3.2 Model Architecture</span>


<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/imgs/vanilla.png"/>

*Figure 4: Illustration of the vanilla model with a segment length 4, from [Dai et.al, 2019](https://arxiv.org/pdf/1901.02860)*


<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/imgs/transformer-xl.png"/>

*Figure 5: Illustration of the Transformer-XL model with a segment length 4, from [Dai et.al, 2019](https://arxiv.org/pdf/1901.02860)*

For model architecture, we utilize the huggingface implementation of [Transformer-XL](https://arxiv.org/pdf/1901.02860), which leverage a **segment-level recurrence mechanism** and a **relative positional encoding scheme** to enable the modeling of longer context.

```python
class Model(nn.Module):
    def __init__(self, checkpoint, is_training=False):
        super(Model, self).__init__()
        # load dictionary
        self.dictionary_path = checkpoint
        self.checkpoint_path = checkpoint
        self.event2word, self.word2event = pickle.load(open(self.dictionary_path, 'rb'))
        # model settings
        self.x_len = 512
        self.mem_len = 512
        self.n_layer = 12
        self.d_embed = 512
        self.d_model = 512
        self.dropout = 0.1
        self.n_head = 8
        self.d_head = self.d_model // self.n_head
        self.d_ff = 2048
        self.n_token = len(self.event2word)
        self.learning_rate = 0.0002
        # load model

        self.configuration = TransfoXLConfig(
            #...concrete config ommitted
        )
         
        # Initializing a model (with random weights) from the configuration
        self.xl = TransfoXLModel(self.configuration)
        self.drop = nn.Dropout(p=self.dropout)
        self.linear = nn.Linear(self.d_embed, self.n_token)

    def forward(self, x):
        outputs = self.xl(input_ids = x)
        output = self.drop(outputs['last_hidden_state']) # dropout
        output_logit = self.linear(output)
        return output_logit
```

### <span style="font-family:serif; font-size:20px;"> 2.3.3 Training Loop</span>


Each epoch, the model trains on batches, periodically validates on a held-out set, saves the best checkpoint, and stops early if validation does not improve for several epochs.

```python
for epoch in range(start_epoch, epochs+1):
    # Training phase
    single_epoch = []
    for i in tqdm(train_dataloader, desc=f"Epoch {epoch} Training"):
        x = i[:, 0, :].to(device).long()  
        y = i[:, 1, :].to(device).long() 
            
        output_logit = model(x)
        # output_logit = (batch, 512, vocab_size)
            
        loss = nn.CrossEntropyLoss()(output_logit.permute(0,2,1), y)
        loss.backward()
        single_epoch.append(loss.to('cpu').mean().item())
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
    # Calculate training NLL for this epoch
    train_epoch_nll = np.array(single_epoch).mean()
    losses.append(train_epoch_nll)
    train_nlls.append(train_epoch_nll)
        
    print(f'>>> Epoch: {epoch}, Train NLL: {train_epoch_nll:.5f}')
        
    # Validation phase
    if epoch % opt.val_freq == 0 or epoch == epochs:
        print(f"Calculating validation NLL for epoch {epoch}...")
        val_nll, val_perplexity, val_tokens = calculate_nll(model, val_dataloader, device, max_batches=50)
        val_nlls.append(val_nll)
            
        print(f'>>> Epoch: {epoch}, Validation NLL: {val_nll:.5f}, Perplexity: {val_perplexity:.2f}')
        print(f'>>> Evaluated on {val_tokens} tokens')
            
        # Early stopping check
        if val_nll < best_val_nll:
            best_val_nll = val_nll
            patience_counter = 0
            print(f'>>> New best validation NLL: {best_val_nll:.5f}')
            # Save best model
            best_model_path = './checkpoints/best_model.pkl'
            torch.save({
                    'epoch': epoch,
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'scheduler': scheduler.state_dict(),
                    'train_nll': train_epoch_nll,
                    'val_nll': val_nll,
                    'best_val_nll': best_val_nll,
                    'patience_counter': patience_counter,
                    'train_nlls': train_nlls,
                    'val_nlls': val_nlls,
                }, best_model_path)
            print(f'>>> Best model saved to {best_model_path}')
        else:
            patience_counter += 1
            print(f'>>> No improvement. Patience: {patience_counter}/{opt.early_stop_patience}')
                
            if patience_counter >= opt.early_stop_patience:
                print(f'>>> Early stopping triggered after {patience_counter} epochs without improvement')
                break
```

### <span style="font-family:serif; font-size:20px;"> 2.3.4 Inference</span>

During the inference stage, we autoregressively apply temperature sampling and concat the output token to input for next iteration.

```python
def temperature_sampling(logits, temperature, topk):
        # probs = np.exp(logits / temperature) / np.sum(np.exp(logits / temperature))
        logits = torch.Tensor(logits)
        probs = nn.Softmax(dim=0)(logits / temperature)
        probs = np.array(probs)
        if topk == 1:
            prediction = np.argmax(probs)
        else:
            sorted_index = np.argsort(probs)[::-1]
            candi_index = sorted_index[:topk]
            candi_probs = [probs[i] for i in candi_index]
            # normalize probs
            candi_probs /= sum(candi_probs)
            # choose by predicted probs
            prediction = np.random.choice(candi_index, size=1, p=candi_probs)[0]
        return prediction
```

# <span style="font-family:serif; font-size:28px;"> 3. Evaluation</span>


## <span style="font-family:serif; font-size:24px;"> 3.1 Context</span>


**Evaluation Metrics:**

Following prior research on music generation ([Huang et al. 2018](https://arxiv.org/pdf/1809.04281); [Novack et al. 2024](https://arxiv.org/abs/2401.12179)), we adopt a range of objective metrics to evaluate our method, which fall into three main categories:

- Learning-related metrics: **Negative log-likelihood (NLL)** on the validation set, which measures how well the model predicts musical events.

- Model-based metrics: **[Fréchet Audio Distance (FAD)](https://arxiv.org/abs/1812.08466)**, which quantifies the similarity between the embeddings of generated and reference audio, providing a perceptual measure of generation quality.

Due to resource constraints, we do not conduct subjective evaluation.

**The properties of a “good” output:** A good output in unconditional music generation should exhibit musical coherence and structure, stylistic consistency with the training data, diversity, and creativity. The music should follow logical progressions in melody, harmony, and rhythm, and sound natural and engaging to human listeners, even though it is not conditioned on any specific prompt or input.

**Relationship between the objective being optimized by your model and the evaluation metrics:** The model is optimized for sequence likelihood (e.g., negative log-likelihood), encouraging statistical similarity to the training data. However, low perplexity does not always ensure musical quality, so we also use perceptual measures like FAD to measure similarity to reference data. Ultimately, subjective evaluation remains necessary for a comprehensive assessment.



<!-- What is the relationship between the objective being optimized by
your model (e.g. perplexity) and musical properties (e.g. does it follow harmonic “rules”?) versus subjective properties? -->

## <span style="font-family:serif; font-size:24px;"> 3.2 Baseline Choices and Evaluation Protocol</span>

**Baselines**

We adopt two next-token prediction models, **Markov Chain and LSTM**, as baselines for comparison, as introduced in the previous section.

<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/imgs/LSTM_Cell.png" width="500"></img><img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/imgs/mc.png" width="500"></img>

*Figure 6: illustration of LSTM (left) and Markov Chain (right)*

**Evaluation Protocol**

For each model, we select the checkpoint with the best validation negative log-likelihood and generate 500 samples. These samples are evaluated against the [POP909](https://arxiv.org/abs/2008.07142) dataset as the reference, using Fréchet Audio Distance (FAD).

**Reference Dataset Statistics**

The POP909 dataset underwent extensive preprocessing to maximize its value for music research. Specifically, beat and chord annotations were extracted from MIDI and audio files using established music information retrieval algorithms from prior work. Key (tonality) annotations were also computed using existing MIR techniques, while tempo curves were carefully hand-labeled to ensure accuracy. By combining manual annotation with automated extraction methods, the dataset provides comprehensive and reliable musical features for each song.

It is important to note that **POP909 was collected independently from POP1K7**, which serves as our training data. As a result, POP909 is well-suited for use as a reference dataset in our evaluation, given that it covers the same genre.

The statistics is presented below:

| Feature            | Mean     | Median   | Min     | Max     | Std      | N   |
|--------------------|----------|----------|---------|---------|----------|-----|
| pitch_density      | 0.0231   | 0.0221   | 0.0079  | 0.1429  | 0.0084   | 909 |
| pitch_range        | 53.3465  | 53.0000  | 29.0000 | 80.0000 | 6.9464   | 909 |
| avg_pitch_interval | 6.5809   | 6.4766   | 3.4998  | 11.0428 | 0.8275   | 909 |
| note_density       | 6.9075   | 6.6670   | 3.4437  | 16.0498 | 1.6247   | 909 |
| avg_ioi            | 0.1505   | 0.1489   | 0.0622  | 0.2885  | 0.0311   | 909 |


<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/pop909_analysis/avg_ioi.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/pop909_analysis/avg_pitch_interval.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/pop909_analysis/note_density.png" width="350"/>
<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/pop909_analysis/pitch_density.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/pop909_analysis/pitch_range.png" width="350"/> 

*Figure 7: Music Feature Distribution of  POP909.*

Figure 8 shows the distributional differences between POP1K7 and POP909. While there is a notable difference in the average pitch interval, the distributions of most other features largely overlap. This suggests that these feature-based metrics are generally reasonable for evaluation. Nevertheless, perceptual metrics such as FAD remain important for a more thorough and comprehensive assessment.

<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/train_ref_diff/avg_ioi.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/train_ref_diff/avg_pitch_interval.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/train_ref_diff/note_density.png" width="350"/>
<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/train_ref_diff/pitch_density.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/train_ref_diff/pitch_range.png" width="350"/> 

*Figure 8: Music Feature Distributional Comparison between POP909 and POP1K7.*

## <span style="font-family:serif; font-size:24px;"> 3.3 Codes</span>

## <span style="font-family:serif; font-size:22px;"> 3.3.1 Implementation</span>

**Negative Log Likelihood**

The average negative log-likelihood (NLL) per token is defined as:

$$
\mathrm{NLL} = -\frac{1}{N} \sum_{i=1}^{N} \log p_\theta(y_i \mid x_i)
$$

where $N$ is the number of tokens, $y_i$ is the target token, and $p_\theta(y_i \mid x_i)$ is the predicted probability from the model.


```python
def calculate_nll(model, dataloader, device, max_batches=None):
    """Calculate negative log-likelihood on validation/test set"""
    model.eval()
    total_nll = 0.0
    total_tokens = 0
    batch_count = 0
    
    with torch.no_grad():
        for i, batch in enumerate(tqdm(dataloader, desc="Calculating NLL")):
            if max_batches and i >= max_batches:
                break
                
            x = batch[:, 0, :].to(device).long() 
            y = batch[:, 1, :].to(device).long()
            
            output_logit = model(x)
            
            # Calculate cross-entropy loss (which is negative log-likelihood)
            loss = nn.CrossEntropyLoss(reduction='sum')(output_logit.permute(0,2,1), y)
            
            total_nll += loss.item()
            total_tokens += y.numel()  # Total number of tokens
            batch_count += 1
    
    avg_nll = total_nll / total_tokens if total_tokens > 0 else float('inf')
    perplexity = torch.exp(torch.tensor(avg_nll)).item()
    
    model.train()  # Set back to training mode
    return avg_nll, perplexity, total_tokens
```

**Fréchet Audio Distance**

Given two sets of embeddings with means $\mu_1, \mu_2$ and covariances $\Sigma_1, \Sigma_2$, the Fréchet distance is:

$$
\mathrm{FAD} = \|\mu_1 - \mu_2\|^2 + \mathrm{Tr}\left(\Sigma_1 + \Sigma_2 - 2(\Sigma_1 \Sigma_2)^{1/2}\right)
$$


```python
def extract_clap_embeddings(audio_paths, processor, model, device, bs=8):
    """Return np.array shape (N, embed_dim)"""
    embeddings = []
    for i in tqdm(range(0, len(audio_paths), bs), desc="Embedding"):
        batch_files = audio_paths[i:i+bs]
        batch_waveforms = [load_audio(path) for path in batch_files]
        inputs = processor(audios=batch_waveforms, return_tensors="pt", sampling_rate=48000, padding=True)
        with torch.no_grad():
            input_tensor = inputs["input_features"].to(device)
            output = model.get_audio_features(input_tensor)
            emb = output.cpu().numpy()
        embeddings.append(emb)
    embeddings = np.concatenate(embeddings, axis=0)
    return embeddings

def compute_fad(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Fréchet distance for two Gaussians."""
    covmean, _ = sqrtm(sigma1.dot(sigma2), disp=False)
    # numerical stability
    if not np.isfinite(covmean).all():
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = sqrtm((sigma1 + offset).dot(sigma2 + offset))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    diff = mu1 - mu2
    return diff.dot(diff) + np.trace(sigma1 + sigma2 - 2*covmean)
```


## <span style="font-family:serif; font-size:22px;"> 3.3.2 Evaluation Result</span>

**Summary**

| Model        | Val NLL  | FAD      |
|--------------|:--------:|:--------:|
| Markov Chain | 2.9434     | 0.1398   |
| LSTM         | 1.2968    |  0.1145  |
| Pop Music Transformer | **1.2154** |  **0.0822**   |




**Analysis of Generated Samples**

We also plot the distributions of musical features for the generated MIDI files produced by each method. The differences between these distributions and those of the reference set are quantified using KL divergence and Wasserstein distance.

The KL divergence between two discrete distributions $P$ and $Q$ is:

$$
D_{KL}(P \,\|\, Q) = \sum_{i} P(i) \log \frac{P(i)}{Q(i)}
$$

For 1-dimensional distributions $P$ and $Q$, the Wasserstein (first-order) distance is:

$$
W(P, Q) = \int_{-\infty}^{\infty} | F_P(x) - F_Q(x) |\, dx
$$

where $F_P(x)$ and $F_Q(x)$ are the cumulative distribution functions of $P$ and $Q$.

**Markov Chain**


<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/mc_eval/avg_ioi.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/mc_eval/avg_pitch_interval.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/mc_eval/note_density.png" width="350"/>
<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/mc_eval/pitch_density.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/mc_eval/pitch_range.png" width="350"/> 

*Figure 9: Music Feature Distributional Comparison between MC-generated samples and POP909.*

**LSTM**

<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/lstm_eval/avg_ioi.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/lstm_eval/avg_pitch_interval.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/lstm_eval/note_density.png" width="350"/>
<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/lstm_eval/pitch_density.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/lstm_eval/pitch_range.png" width="350"/> 

*Figure 10: Music Feature Distributional Comparison between LSTM-generated samples and POP909.*

**Pop Music Transformer**

<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/popmt_eval/avg_ioi.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/popmt_eval/avg_pitch_interval.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/popmt_eval/note_density.png" width="350"/>
<img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/popmt_eval/pitch_density.png" width="350"/> <img src="https://anonymous.4open.science/r/remi-pytorch-6C7D/popmt_eval/pitch_range.png" width="350"/> 

*Figure 11: Music Feature Distributional Comparison between Transformer-generated samples and POP909.*


# <span style="font-family:serif; font-size:28px;"> 4. Related works</span>


## <span style="font-family:serif; font-size:24px;"> 4.1 Previous Dataset Usage</span>


The datasets we use, POP1K7 and POP909, have been widely adopted in previous research.

For example, POP909 has been used in studies on symbolic music classification ([Chou and Chen, 2024](https://arxiv.org/pdf/2107.05223v2)), symbolic music generation ([Choi and Lee, 2023](https://arxiv.org/pdf/2211.00895v2), [Shin et.al, 2022](https://arxiv.org/pdf/2111.04093v2)), and music representation/tokenization ([Fradet et al., 2023](https://arxiv.org/pdf/2301.11975v3)).  POP1K7 is primarily employed for tasks related to symbolic music generation([Zhang et al., 2022](https://arxiv.org/pdf/2211.00222)) and understanding ([Liang et al., 2024](https://arxiv.org/abs/2407.03361)).


## <span style="font-family:serif; font-size:24px;"> 4.2 Other Work on Related Tasks</span>


As discussed in the previous section, piano composition can be formulated as a sequential modeling problem. Accordingly, sequential models have been predominantly adopted for this task.

[Oore and Simon (a), 2017](https://magenta.tensorflow.org/performance-rnn) and [Oore and Simon (b), 2020](https://arxiv.org/abs/1808.03715) employed recurrent neural networks (RNNs) to compose polyphonic music with expressive timing and dynamics.

[Huang et al., 2018](https://arxiv.org/pdf/1809.04281) introduced a Transformer-based approach with enhanced sequential modeling capabilities for music generation. In a later work, [Huang et al., 2020](https://dl.acm.org/doi/pdf/10.1145/3394171.3413671) proposed the REMI tokenization scheme, further improving upon vanilla Transformer models for symbolic music tasks.

[Muhamed et al., 2021](https://cdn.aaai.org/ojs/16117/16117-13-19611-1-2-20210518.pdf) incorporated adversarial loss from Generative Adversarial Networks (GANs) to improve the ability of Transformer models to generate long-term consistent music sequences.


## <span style="font-family:serif; font-size:24px;"> 4.3 Results Comparison to Related Work</span>


Overall, our results are consistent with findings reported in previous research ([Huang et al., 2018](https://arxiv.org/pdf/1809.04281)), which demonstrate that transformer-based methods offer significant improvements over traditional baselines such as Markov Chain models and RNN-based models in both validation NLL and FAD.